In [13]:
import numpy as np
import pandas as pd 
from scipy.stats import norm

In [14]:
## Heston model

def simulate_heston(S0, v0, r, kappa, theta, sigma, rho, T, N, M):
    dt = T / N
    S = np.zeros((M, N + 1))
    v = np.zeros((M, N + 1))
    S[:, 0] = S0
    v[:, 0] = v0

    for t in range(1, N + 1):
        z1 = np.random.normal(size=M)
        z2 = rho * z1 + np.sqrt(1 - rho**2) * np.random.normal(size=M)


        v[:, t] = np.maximum(v[:, t-1] + kappa * (theta - v[:, t-1]) * dt +
                             sigma * np.sqrt(v[:, t-1] * dt) * z1, 0)
        S[:, t] = S[:, t-1] * np.exp((r - 0.5 * v[:, t-1]) * dt +
                                     np.sqrt(v[:, t-1] * dt) * z2)
        
    return S, v

In [15]:
## Black-Scholes

def black_scholes_price(S, K, T, r, sigma, option_type='call'):
    # Handle degenerate cases
    if T <= 0 or sigma <= 0:
        if option_type == 'call':
            return max(S - K, 0)
        else:
            return max(K - S, 0)
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == 'call':
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    else:
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)


In [16]:
S0 = 100     # Initial stock price
v0 = 0.04    # Initial variance (20% volatility)
r = 0.01     # Risk-free rate
kappa = 2.0  # Mean reversion rate
theta = 0.04 # Long-run variance
sigma = 0.5  # Vol of vol
rho = -0.7   # Correlation between price and volatility
T = 1        # Total time (1 year)
N = 252      # Trading days per year
M = 1000     # Number of simulated paths

S_paths, v_paths = simulate_heston(S0, v0, r, kappa, theta, sigma, rho, T, N, M)

In [17]:
S_paths, v_paths

(array([[100.        , 101.17061004, 101.9355591 , ..., 106.17820561,
         106.5715785 , 106.49074206],
        [100.        , 100.215584  , 101.48042871, ...,  86.12270178,
          87.10179259,  86.00390333],
        [100.        , 100.84059783,  99.1871034 , ...,  71.1677692 ,
          71.66113173,  70.09193345],
        ...,
        [100.        , 100.26411751, 101.66719237, ..., 119.42935926,
         119.34219017, 120.07523763],
        [100.        ,  98.26648262,  99.05045075, ..., 102.2253038 ,
          99.77368981,  99.27424257],
        [100.        ,  99.67912802,  98.49258088, ...,  95.4249015 ,
          94.51303206,  94.1999206 ]], shape=(1000, 253)),
 array([[0.04      , 0.03165946, 0.02952081, ..., 0.05843695, 0.05567292,
         0.05528628],
        [0.04      , 0.04148149, 0.03649746, ..., 0.07497612, 0.07625524,
         0.08226162],
        [0.04      , 0.03895506, 0.04681162, ..., 0.06397391, 0.06013783,
         0.0788386 ],
        ...,
        [0.04    

In [18]:
# Define parameters
snapshot_days = [21, 63, 126, 189, 252]  # simulate monthly market snapshots
strikes = np.arange(80, 121, 5)          # range of strike prices
expiries = [30, 60, 90]                  # expiries in days
option_types = ['call', 'put']

# Store results
option_data = []

# Loop over selected snapshots
for day in snapshot_days:
    for path_index in range(S_paths.shape[0]):
        S = S_paths[path_index, day]
        v = v_paths[path_index, day]
        sigma_t = np.sqrt(v)
        for K in strikes:
            for expiry in expiries:
                T_exp = expiry / 252  # convert days to years
                for opt_type in option_types:
                    price = black_scholes_price(S, K, T_exp, r, sigma_t, option_type=opt_type)
                    option_data.append({
                        'path_id': path_index,
                        'snapshot_day': day,
                        'spot_price': S,
                        'strike': K,
                        'expiry_days': expiry,
                        'option_type': opt_type,
                        'price': price,
                        'iv': sigma_t
                    })

In [19]:
df_options = pd.DataFrame(option_data)
df_options.round(2)

,path_id,snapshot_day,spot_price,strike,expiry_days,option_type,price,iv
0,0,21,100.53,80,30,call,20.62,0.18
1,0,21,100.53,80,30,put,0.00,0.18
2,0,21,100.53,80,60,call,20.73,0.18
3,0,21,100.53,80,60,put,0.01,0.18
4,0,21,100.53,80,90,call,20.86,0.18
...,...,...,...,...,...,...,...,...
269995,999,252,94.20,120,30,put,25.66,0.19
269996,999,252,94.20,120,60,call,0.01,0.19
269997,999,252,94.20,120,60,put,25.53,0.19
269998,999,252,94.20,120,90,call,0.07,0.19


In [20]:
print("Generated options:", df_options.shape[0])
df = pd.read_csv("synthetic_option_chain.csv")


Generated options: 270000
